In [ ]:
#default_exp utils

# Basic Utils

> Basic utility functions across sac_stac.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import pandas as pd

In [ ]:
#export
import os
import zipfile
import time
from glob import glob
from urllib.parse import urlparse


import boto3
import gdal
from pystac import STAC_IO

from sedas_pyapi.sedas_api import SeDASAPI
from sedas_pyapi.bulk_download import SeDASBulkDownload

## Sedas

Probably best if we start by logging into sedas to be able to access datasets in their current format.

In [ ]:
#export
def sedas_client():
    "Log into sedas api."
    sedas = SeDASAPI(os.getenv('SEDAS_USERNAME'), os.getenv('SEDAS_PWD'))
    sedas.base_url="https://geobrowsertest.satapps.org/api/"
    return sedas

In [ ]:
sedas_client()

should inc. test for env vars...?

And we'll want to find datasets from different *Collections*, which sedas expects as *groups*. Will eventually **need to add test endpoint** to be able to access NovaSAR datasets.

In [ ]:
#export
def sedas_get_collections():
    sedas = sedas_client()
    result_groups = sedas.list_sensor_groups()
    groups = []
    for i in range(0, len(result_groups)):
        groups.append(result_groups[i]['name'])
    return f"Available groups are: {', '.join(groups)}"

In [ ]:
sedas_get_collections()

'Available groups are: Cosmo-SkyMed, SPOT, Pleiades, S1, S2, AIRSAR'

In [ ]:
#export
def sedas_find_datasets(wkt, startDate, endDate, collection):
    optical_groups = ['S2','Pleiades','SPOT']
    sar_groups = ['S1','Cosmo-SkyMed','AIRSAR']
    sedas = sedas_client()
    if collection in optical_groups: 
        res = sedas.search_optical(wkt, startDate, endDate, source_group=collection) 
    elif collection in sar_groups:
        res = sedas.search_sar(wkt, startDate, endDate, source_group=collection)
    return res

For example for a small area over Oxford we can find...

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products'])

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,c0ec9a5f12356e87bc910ddbc49dbb76,Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.654728 51.309517,-1.345414 51.3081...",2018-10-24T11:17:22Z,...,4.825125e+08,1.0,5.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'cf...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,94cc7887414be0912de7ca44288f79da,Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-2.153852 51.603313,-1.841533 51.6033...",2018-09-29T11:10:08Z,...,1.565260e+08,2.0,22.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
2,6ac7da92a92161839c0bf29ab1b878d3,Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.903588 51.628784,-1.592515 51.6281...",2018-09-29T11:10:00Z,...,1.796598e+08,10.0,100.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'e6...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
3,dd08a178209ed51cc57f5caef1e1ad7c,Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.401923 51.155725,-1.101068 51.1538...",2018-09-02T11:17:34Z,...,8.211908e+08,1.0,1.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'a5...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
4,97528406bc14d1c94ef22d20b4b26c7a,Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.576165 51.728406,-1.265721 51.7268...",2018-07-24T11:24:30Z,...,4.256422e+08,11.0,46.0,1.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '77...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
5,3e8d4c9096b3ba5c7126499e442424f5,Pleiades_UKSA87_SO18034613-87-01_DS_PHR1A_2018...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.413484 51.469752,-1.089638 51.4676...",2018-06-29T11:18:17Z,...,9.264473e+08,34.0,67.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '7f...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
6,b5ff24c70eba28f3ec7292c356fb27b0,Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.928964 51.516819,-1.63821 51.51628...",2018-02-25T11:21:25Z,...,2.385591e+08,12.0,94.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
7,92fe8121944a5267861d31597afb016a,Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.606218 51.306711,-1.294645 51.3052...",2018-02-24T11:27:55Z,...,6.639221e+08,11.0,29.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'ef...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


We can do a single download.

In [ ]:
# #export
# def sedas_download(sedas_res_dict, down_zip, sedas=None):
#     "Use product dict to download product zip."
#     if not sedas:
#         sedas = sedas_client()
#     sedas.download(sedas_res_dict, down_zip)

In [ ]:
#export
def sedas_download(sedas_res_dicts, down_dir, sedas=None):
    "Use product dicts list to download into a dir."
    if not sedas:
        sedas = sedas_client()
    downloader = SeDASBulkDownload(sedas, down_dir, parallel=2)
    downloader.add(sedas_res_dicts)
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Downloading")
    while not downloader.is_done():
        time.sleep(5)
    downloader.shutdown()
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Downloaded")

In [ ]:
# tbd example

In [ ]:
#export
def sedas_extract(down_zip, scene_dir):
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Extracting {down_zip}")
    with zipfile.ZipFile(down_zip, 'r') as zip_ref:
        zip_ref.extractall(scene_dir)
    if os.path.exists(scene_dir):
        os.remove(down_zip)
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Extracted to {scene_dir}")

In [ ]:
# tbd example

## Cloud-Optimised Formatting

Once we have access to the datasets, we will want to convert raster images of any format into a default COG format. (It helps that COGs are now an official [gdal driver](https://gdal.org/drivers/raster/cog.html).

In [ ]:
#export
def convert2cog(img_path, cog_path, band):
    """
    Convert gdal raster into COG with default settings.
    Considering lzw compression.
    See https://www.cogeo.org/developers-guide.html.
    """
    # translate into new cog file
    kwargs = {
        'format': 'COG',
#         'creationOptions' : ['COMPRESS=LZW'],
        'bandList': [band]
    }
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Starting conversion: {img_path}.")
    ds = gdal.Translate(cog_path, img_path, **kwargs)
    ds = None
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Conversion complete: {os.path.exists(cog_path)}.")

In [ ]:
#export
def mosaic2cog(imgs, out_cog, band):
    """
    Mosaic imgs into cog.
    Usual vrt assumptions.
    """
    tmp_vrt = f"{out_cog[:-4]}_mosaic.tif"
    # mosaic into vrt
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Mosaicing band {band} imgs: {imgs}")
    ds = gdal.BuildVRT(tmp_vrt, imgs, bandList=[band])
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Mosaicd {tmp_vrt}")
    # write vrt mosaic to cog
    convert2cog(ds, out_cog, 1)
    ds = None

    if os.path.exists(tmp_vrt):
        os.remove(tmp_vrt)

mosaic currently specific to pleiades ([:-13] for slicing name). Needs to be generalised a little more

In [ ]:
#export
def cogmosaicbands(img_paths, numbands, basename):
    """
    Convert bands from img_paths to cog, output 
    basename_band#.tif. Mosaic if >1 img paths.
    """
    if len(img_paths) == 1:
        for b in range(1, numbands+1):
            convert2cog(img_paths[0], f"{basename}_band{b}.tif", b)
        for f in glob(f"{img_paths[0][:-4]}*"):
            os.remove(f)
    elif len(img_paths) > 1:
        for b in range(1, numbands+1):
            mosaic2cog(img_paths[0], f"{basename}_band{b}.tif", b)
        for img_path in img_paths:
            for f in glob(f"{img_path[:-4]}*"):
                os.remove(f)

## Object Storage

In [ ]:
#export
def s3_create_client(s3_bucket):
    """
    Create and set up a connection to S3
    :param s3_bucket:
    :return: the s3 client object.
    """
    access = os.getenv("AWS_ACCESS_KEY_ID")
    secret = os.getenv("AWS_SECRET_ACCESS_KEY")

    session = boto3.Session(
        access,
        secret,
    )
    endpoint_url = os.getenv("AWS_S3_ENDPOINT_URL")

    if endpoint_url is not None:
        s3 = session.resource('s3', endpoint_url=endpoint_url)
    else:
        s3 = session.resource('s3', region_name='eu-west-2')

    bucket = s3.Bucket(s3_bucket)

    if endpoint_url is not None:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret,
            endpoint_url=endpoint_url
        )
    else:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret
        )
    return s3_client, bucket

We can use this to connect to any S3-like object storage end point. Assumes env vars.

In [ ]:
#export
gb = 1024 ** 3

In [ ]:
#export
def s3_single_upload(in_path, s3_path, s3_bucket):
    """
    put a file into S3 from the local file system.
    :param in_path: a path to a file on the local file system
    :param s3_path: where in S3 to put the file.
    :return: None
    """
    # prep session & creds
    s3_client, bucket = s3_create_client(s3_bucket)

    # Ensure that multipart uploads only happen if the size of a transfer is larger than
    # S3's size limit for non multipart uploads, which is 5 GB. we copy using multipart 
    # at anything over 4gb
    transfer_config = boto3.s3.transfer.TransferConfig(multipart_threshold=2 * gb,
                                                       max_concurrency=10,
                                                       multipart_chunksize=2 * gb,
                                                       use_threads=True)
    s3_client.upload_file(in_path, bucket.name, s3_path)

In [ ]:
#export
def get_rel_dir_s3_paths(local_dir, s3_dir):
    """
    returns local path, remote path pair list.
    """  
    paths = []
    for subdir, dirs, files in os.walk(local_dir):
        for file in files:
            full_path = os.path.join(subdir, file)
            paths.append([ full_path, s3_dir + local_dir.split('/')[-2] + full_path[len(local_dir):] ])
    return paths

In [ ]:
#export
def s3_upload_dir(in_dir, s3_bucket, s3_dir):
    """
    Upload all items in directory, inc. dir.
    """
    paths = get_rel_dir_s3_paths(in_dir, s3_dir)
    upload_list = [(in_path, out_path, s3_bucket)
                   for in_path, out_path in paths]
    for i in upload_list:
        s3_single_upload(i[0], i[1], i[2])

In [ ]:
#export
def s3_list_objects_paths(s3_bucket, prefix):
    """List of paths only returned, not full object responses"""
    client, bucket = s3_create_client(s3_bucket)
    
    return [e['Key'] for p in client.get_paginator("list_objects_v2").paginate(Bucket=s3_bucket, Prefix=prefix) for e in p['Contents']]


In [ ]:
#export
def s3_list_objects(s3_bucket, prefix):
    # prep session & creds
    client, bucket = s3_create_client(s3_bucket)
    response = client.list_objects_v2(Bucket=s3_bucket, Prefix=prefix)

    return response

## General

In [ ]:
#export
def clean_up(work_dir):
    # TODO: sort out logging changes...
    gc.collect()
    shutil.rmtree(work_dir)
    pass

## STAC

We can amend the default STAC I/O tools to work with our object storage.

First up is to be able to create a uri given an endpoint, bucket name and object path.

In [ ]:
#export
def create_uri(s3_path, s3_bucket='public-eo-data', endpoint_url=os.getenv('AWS_S3_ENDPOINT_URL')):    
    return f"{endpoint_url}/{s3_bucket}/{s3_path}"

Amend the reading method.

In [ ]:
#export
def my_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme == 's3':
        bucket = parsed.netloc
        key = parsed.path[1:]
        s3 = boto3.resource('s3')
        obj = s3.Object(bucket, key)
        return obj.get()['Body'].read().decode('utf-8')
    else:
        return STAC_IO.default_read_text_method(uri)

And the writing method.

In [ ]:
#export
def my_write_method(uri, txt):
    parsed = urlparse(uri)
    if parsed.scheme == 's3':
        bucket = parsed.netloc
        key = parsed.path[1:]
        s3 = boto3.resource("s3")
        s3.Object(bucket, key).put(Body=txt)
    else:
        STAC_IO.default_write_text_method(uri, txt)

Then allow us to overwrite the defaults.

In [ ]:
#export
def pystac_setIO():
    STAC_IO.read_text_method = my_read_method
    STAC_IO.write_text_method = my_write_method

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.ipynb.
Converted 01_pleiades.ipynb.
Converted index.ipynb.
